# Lab Sheet 3 - Solution: Extracting Word Frequency Vectors with Spark

These tasks are for working in the lab session and during the week. We will use the same data as last week (19 files in './City-Data-Science/library/') and use some more RDD functions. We will apply two different approaches to create and use fixed size vectors.

First update the repo.

In [10]:
!git clone https://github.com/tweyde/City-Data-Science.git
%cd City-Data-Science/
!git pull https://github.com/tweyde/City-Data-Science.git
%cd ..

fatal: destination path 'City-Data-Science' already exists and is not an empty directory.
/Users/tweyde/Documents/CityUni/teaching/2017-18/Big Data/labs/lab3/City-Data-Science
From https://github.com/tweyde/City-Data-Science
 * branch            HEAD       -> FETCH_HEAD
Already up-to-date.
/Users/tweyde/Documents/CityUni/teaching/2017-18/Big Data/labs/lab3


Here is code from last week that we run first, and then extend. 

In [11]:
import re 

def stripFinalS( word ):
    word = word.lower() # lower case
    if len(word) >0 and word[-1] == 's': # check for final letter
        return word[:-1]
    else:
        return word
    
def splitFileWords(filenameContent): # your splitting function
    f,c = filenameContent # split the input tuple  
    fwLst = [] # the new list for (filename,word) tuples
    wLst = re.split('\W+',c) # <<< now create a word list wLst
    for w in wLst : # iterate through the list
        fwLst.append((f,stripFinalS(w))) # and append (f,w) to the 
    return fwLst #return a list of (f,w) tuples 

from pyspark import SparkContext

sc = SparkContext.getOrCreate()

dirPath = './City-Data-Science/library/' #  path
ft_RDD = sc.wholeTextFiles(dirPath) #<<< add code to create an RDD with wholeTextFiles
fnt_RDD = ft_RDD.map(lambda ft: (re.split('[/\.]',ft[0])[-2],ft[1])) # just take filename, 
                                                # drop path and extension for readability
fw_RDD1 = fnt_RDD.flatMap(splitFileWords)
fw_RDD = fw_RDD1.filter(lambda fw: len(fw[1])>0 and fw[1] not in ['project','gutenberg', 'ebook'])  
fw_RDD.take(3)

[('emma', 'the'), ('emma', 'of'), ('emma', 'emma')]

## 1) Warm-up
Let's start with a few small tasks, to become more fluent with RDDs and lambda expressions.

a) Count the number of documents.
b) Determine the number distinct words in total (the vocabulary size) using RDD.distinct(). This involves removing the fs from the (f,w) pairs and geting getting the RDD size (with RDD.count()). 
c) Get the number of words (including repeated ones) per book. 
d) Determine the number of distinct words per book. This involves determining the disting (f,w) pairs, geting a list of words per file, and getting the list size.
e) Count the average number of occurences per word per file (words/vocabulary). Use RDD.join() to get both numbers into one RDD. 

In [12]:
# a) Library size
print("Number of documents: ",ft_RDD.count())

# b) Vocabulary size
w_RDD = fw_RDD.map(lambda fw: fw[1])
w_RDDu = w_RDD.distinct()
print('Total vocabulary size: ',w_RDDu.count())
# this can also be programmed in short as follows:
print('Total vocabulary size (v2): ',fw_RDD.values().distinct().count())

# c) words per book
f1_RDD = fw_RDD.map(lambda fw: (fw[0],1)) # swap and wrap (f,w) to (w,1)
fc_RDD = f1_RDD.reduceByKey(add)
print('Words per book: ',fc_RDD.take(3))
# or alternatively (both versions should produce the same output):
print('Words per book (v2): ',fw_RDD.countByKey())

# d) Vocabulary per book
from operator import add
fw_RDDu = fw_RDD.distinct() # get unique (f,w) pairs - i.e. evey word only once per file. I use postfix u to mark 'unique'
f1_RDDu = fw_RDDu.map(lambda fw: (fw[0],1)) # swap and wrap (f,w) to (w,[f])
fcu_RDD = f1_RDDu.reduceByKey(add)
print('Vocabulary per book: ',fcu_RDD.take(3))
# or, again, shorter:
print('Vocabulary per book (v2): ',fw_RDD.distinct().countByKey())

# e) Average occurences of words per book (i.e. words/vocab per book)
f_wv_RDD = fc_RDD.join(fcu_RDD) # join the two RDDs to get (f,(w,v)) tuples
print(f_wv_RDD.take(3)) 
f_awo_RDD = f_wv_RDD.map(lambda f_wv: (f_wv[0],f_wv[1][0]/f_wv[1][1])) # this is the tricky part. 
# Resolve nested tuples in the lambda to get (filename,words/vocab) tuples
print('Average word occurences: ',f_awo_RDD.take(3))
# should look like this [('henry_V', 6.237027812370278), ('king_lear', 7.815661103979461), ('lady_susan', 8.531763947113834)]

Number of documents:  18
Total vocabulary size:  23368
Total vocabulary size (v2):  23368
Words per book:  [('henry_V', 29900), ('king_lear', 30119), ('lady_susan', 26109)]
Words per book (v2):  defaultdict(<class 'int'>, {'emma': 163979, 'hamlet': 34518, 'henry_V': 29900, 'julius_cesar': 22959, 'king_lear': 30119, 'lady_susan': 26109, 'macbeth': 20810, 'mansfield_park': 163509, 'merchant_of_venice': 24772, 'midsummer': 19814, 'northanger_abbey': 77582, 'othello': 30617, 'persuasion': 83680, 'prideandpredjudice': 125212, 'richard_III': 33785, 'romeo_and_juliet': 29538, 'senseandsensibility': 123066, 'tempest': 27891})
Vocabulary per book:  [('henry_V', 4815), ('king_lear', 3892), ('lady_susan', 3097)]
Vocabulary per book (v2):  defaultdict(<class 'int'>, {'emma': 6667, 'hamlet': 4447, 'henry_V': 4815, 'julius_cesar': 2834, 'king_lear': 3892, 'lady_susan': 3097, 'macbeth': 3654, 'mansfield_park': 7492, 'merchant_of_venice': 3686, 'midsummer': 3448, 'northanger_abbey': 5510, 'othello': 4


## 2) Fixed vectors: Reduced vocabulary approach

The first task in this lab is to use a reduced vocabulary, only the stopwords from a list, to make sure that we have a fixed size vector. This is a common approach in stylometry. The problem is that some stopwords might not appear in some documents. We will deal with that by creating an RDD with ((f,w),0) tuples that we then merge with the ((f,w),count) RDD. 

Start by running the code above, then you can add 1s and use reduceByKey(add) like last week to get the counts of the words per filename.

Then, please make sure that all stopwords are present by creating a new RDD that contains the keys of the fw_RDD, i.e. the filenames, using the keys() method of class RDD. Then you can use flatMap to create a [((filname,stopword),0), ...] list, using a list comprehension. The 0s should not be 1s, as we don't want add to add extra counts.
The RDD with ((filename,stopword),0) tuples can then be merged with fw_RDD2 using union(). Then you can count as normal.

In [13]:
from operator import add

stopwlst = ['the','a','in','of','on','at','for','by','i','you','me'] # stopword list
fw_RDD2 = fw_RDD.filter(lambda x: x[1] in stopwlst) # filter, keeping only stopwords

#fw_RDD.keys().collect()
fsw_0_RDD = fw_RDD.keys().flatMap(lambda f: [((f,sw),0) for sw in stopwlst])
print(fsw_0_RDD.take(3))

fw_1_RDD = fw_RDD2.map(lambda x: (x,1))  #<<< change (f,w) to ((f,w),1)
print(fw_1_RDD.take(3))

fw_10_RDD = fw_1_RDD.union(fsw_0_RDD)
print(fw_10_RDD.take(3))

fw_c_RDD = fw_10_RDD.reduceByKey(add) #<<< count the words
print(fw_c_RDD.take(3))

[(('emma', 'the'), 0), (('emma', 'a'), 0), (('emma', 'in'), 0)]
[(('emma', 'the'), 1), (('emma', 'of'), 1), (('emma', 'by'), 1)]
[(('emma', 'the'), 1), (('emma', 'of'), 1), (('emma', 'by'), 1)]
[(('emma', 'the'), 5380), (('emma', 'by'), 591), (('emma', 'you'), 2068)]


## 3) Creating sorted lists

As a next step, map the `((filename,word),count)` to `( filename, [ (word, count) ])` using the function `reGrpLst` to regroup and create a list. 

Then sort the [(word,count),...] lists in the values (i.e. 2nd part of the tuple) with the the words as keys. Have a [look at the Python docs](https://docs.python.org/3.5/library/functions.html?highlight=sorted#sorted) for how to do this. Hint: use a lambda that extracts the words as the key, e.g. `sorted(f_wdL[1], key = lambda wc: ... )`.   

In [14]:
def reGrpLst(fw_c): # we get a nested tuple
    fw,c = fw_c     # split the outer tuple
    f,w = fw        # split the inner tuple
    return (f,[(w,c)]) # return (f,[(w,c)]) structure. Can be used verbatim, if your variable names match.

fw_RDD
f_wcL_RDD = fw_c_RDD.map(reGrpLst) 
f_wcL2_RDD = f_wcL_RDD.reduceByKey(add) #<<< create [(w,c), ... ,(w,c)] lists per file 
f_wcLsort_RDD = f_wcL2_RDD.map(lambda f_wcL: (f_wcL[0], sorted(f_wcL[1],key=lambda x: x[0]))) #<<< sort the lists
print(f_wcLsort_RDD.take(3))
f_wVec_RDD = f_wcLsort_RDD.map(lambda f_wc: (f_wc[0],[float(c) for (w,c) in f_wc[1]])) #<<< remove the words and convert the numbers to floats
f_wVec_RDD.take(3)

[('lady_susan', [('a', 611), ('at', 161), ('by', 152), ('for', 262), ('i', 1106), ('in', 402), ('me', 200), ('of', 787), ('on', 140), ('the', 784), ('you', 353)]), ('macbeth', [('a', 395), ('at', 64), ('by', 74), ('for', 142), ('i', 581), ('in', 227), ('me', 119), ('of', 427), ('on', 76), ('the', 765), ('you', 272)]), ('merchant_of_venice', [('a', 646), ('at', 75), ('by', 131), ('for', 254), ('i', 976), ('in', 319), ('me', 260), ('of', 535), ('on', 81), ('the', 938), ('you', 507)])]


[('lady_susan',
  [611.0,
   161.0,
   152.0,
   262.0,
   1106.0,
   402.0,
   200.0,
   787.0,
   140.0,
   784.0,
   353.0]),
 ('macbeth',
  [395.0, 64.0, 74.0, 142.0, 581.0, 227.0, 119.0, 427.0, 76.0, 765.0, 272.0]),
 ('merchant_of_venice',
  [646.0, 75.0, 131.0, 254.0, 976.0, 319.0, 260.0, 535.0, 81.0, 938.0, 507.0])]

## 4) Clustering

Now we have feature vectors of fixed size, we can use KMeans as provided by Spark.

The files in our library are by two authors. After clustering, check if the cluters reflect authorship:

WILLIAM SHAKESPEARE: 
merchant_of_venice, 
richard_III, 
midsummer,
tempest,
romeoandjuliet,
othello,
henry_V,
macbeth,
king_lear,
julius_cesar,
hamlet

JANE AUSTEN
mansfield_park,
emma,
northanger_abbey,
lady_susan,
persuasion,
prideandpredjudice,
senseandsensibility

In [15]:

from math import sqrt

from pyspark.mllib.clustering import KMeans #, KMeansModel

#print('f_wVec_RDD.take(2): ', f_wVec_RDD.take(1))
wVec_RDD = f_wVec_RDD.map(lambda f_wcl: f_wcl[1]) # strip the filenames
#print(wVec_RDD.collect())

# Build the model (cluster the data)
clusterModel = KMeans.train(wVec_RDD, 2, maxIterations=10, initializationMode="random")

# Assign the files to the clusters
fc_RDD = f_wVec_RDD.map(lambda fv: (fv[0],clusterModel.predict(fv[1])))
for s in fc_RDD.collect():
    print(s)

# Evaluate clustering by computing Within Set Sum of Squared Errors
def error(point):
    center = clusterModel.centers[clusterModel.predict(point)]
    return sqrt(sum([x**2 for x in (point - center)]))

WSSSE = wVec_RDD.map(lambda point: error(point)).reduce(lambda x, y: x + y)
print("Within Set Sum of Squared Error = " + str(WSSSE))


('lady_susan', 1)
('macbeth', 1)
('merchant_of_venice', 1)
('othello', 1)
('persuasion', 0)
('emma', 0)
('julius_cesar', 1)
('mansfield_park', 0)
('richard_III', 1)
('tempest', 1)
('henry_V', 1)
('king_lear', 1)
('midsummer', 1)
('northanger_abbey', 0)
('prideandpredjudice', 0)
('romeo_and_juliet', 1)
('senseandsensibility', 0)
('hamlet', 1)
Within Set Sum of Squared Error = 15118.000047182637


## 5) Alternative approach: feature hashing

Instead of the previous appraoch, we now use feature hashing, as done last week.

In [18]:
def hashing_vectorizer(word_count_list, N):
     v = [0] * N  # create fixed size vector of 0s
     for word_count in word_count_list:          word,count = word_count 	# unpack tuple
         h = hash(word) # get hash value
         v[h % N] = v[h % N] + count # add count
     return v 	# return hashed word vector

from operator import add

N = 10

# we use fw_RDD from the beginning with all the words, not just stopwords
fw_1_RDD = fw_RDD.map(lambda x: (x,1))  #<<< change (f,w) to ((f,w),1)
fw_c_RDD = fw_1_RDD.reduceByKey(add) #as above
f_wcL_RDD = fw_c_RDD.map(reGrpLst) #as above
f_wcL2_RDD = f_wcL_RDD.reduceByKey(add) #<<< create [(w,c), ... ,(w,c)] lists per file 
f_wVec_RDD = f_wcL2_RDD.map(lambda f_wc: (f_wc[0],hashing_vectorizer(f_wc[1],N)))
print(f_wVec_RDD.take(3))

[('henry_V', [3179, 3045, 2431, 2814, 2946, 3252, 2144, 4707, 1821, 3561]), ('king_lear', [3619, 3106, 2258, 3538, 3393, 2844, 2236, 4138, 2111, 2876]), ('lady_susan', [3448, 2639, 1874, 2737, 2847, 2894, 1994, 3851, 1158, 2667])]


In [19]:
from math import sqrt

from pyspark.mllib.clustering import KMeans #, KMeansModel

#print('f_wVec_RDD.take(2): ', f_wVec_RDD.take(1))
wVec_RDD = f_wVec_RDD.map(lambda f_wcl: f_wcl[1]) # strip the filenames
#print(wVec_RDD.collect())

# Build the model (cluster the data)
clusterModel = KMeans.train(wVec_RDD, 2, maxIterations=10, initializationMode="random")

# Assign the files to the clusters
fc_RDD = f_wVec_RDD.map(lambda fv: (fv[0],clusterModel.predict(fv[1])))
for s in fc_RDD.collect():
    print(s)

# Evaluate clustering by computing Within Set Sum of Squared Errors
def error(point):
    center = clusterModel.centers[clusterModel.predict(point)]
    return sqrt(sum([x**2 for x in (point - center)]))

WSSSE = wVec_RDD.map(lambda point: error(point)).reduce(lambda x, y: x + y)
print("Within Set Sum of Squared Error = " + str(WSSSE))

('henry_V', 1)
('king_lear', 1)
('lady_susan', 1)
('macbeth', 1)
('merchant_of_venice', 1)
('midsummer', 1)
('northanger_abbey', 1)
('othello', 1)
('persuasion', 1)
('prideandpredjudice', 0)
('romeo_and_juliet', 1)
('senseandsensibility', 0)
('emma', 0)
('hamlet', 1)
('julius_cesar', 1)
('mansfield_park', 0)
('richard_III', 1)
('tempest', 1)
Within Set Sum of Squared Error = 90883.15633407317


## 6) Neutralising document length: Normalised vectors

'Lady Susan' ends up reliably in the wrong cluster. A possible explanation is that it is shorter than the other Austen works. Try normalising the word counts, i.e. by dividing by their sum. That takes away the effect of length. What is the effect on the clustering?
    
You can use a list comprehension for the normalisation.

In [21]:
nwVec_RDD = wVec_RDD.map(lambda v: ([c/sum(v) for c in v])) 
# provide a list compresention that normalises the values by the 
print("Normalised vectors: ",nwVec_RDD.take(3))

# Build the model (cluster the data)
clusterModel = KMeans.train(nwVec_RDD, 2, maxIterations=10, initializationMode="random")

# Assign the files to the clusters
fc_RDD = f_wVec_RDD.map(lambda fv: (fv[0],clusterModel.predict(fv[1])))
for s in fc_RDD.collect():
    print(s)



Normalised vectors:  [[0.10632107023411372, 0.10183946488294314, 0.08130434782608696, 0.09411371237458194, 0.09852842809364548, 0.10876254180602006, 0.0717056856187291, 0.1574247491638796, 0.06090301003344482, 0.11909698996655518], [0.12015671171021615, 0.10312427371426675, 0.07496928848899366, 0.11746737939506624, 0.1126531425346127, 0.09442544573193001, 0.07423885255154554, 0.13738835950728776, 0.07008864836149939, 0.09548789800458182], [0.1320617411620514, 0.1010762572293079, 0.07177601593320311, 0.10482975219273048, 0.10904285878432725, 0.1108430043280095, 0.07637213221494504, 0.14749703167490139, 0.044352522118809606, 0.10214868436171436]]
('henry_V', 0)
('king_lear', 0)
('lady_susan', 0)
('macbeth', 0)
('merchant_of_venice', 0)
('midsummer', 0)
('northanger_abbey', 0)
('othello', 0)
('persuasion', 0)
('prideandpredjudice', 0)
('romeo_and_juliet', 0)
('senseandsensibility', 0)
('emma', 0)
('hamlet', 0)
('julius_cesar', 0)
('mansfield_park', 0)
('richard_III', 0)
('tempest', 0)


## 7) Building an index

Starting from the fw_RDD we now start building the index and calculating the IDF values. Since we have the TF values alread, we only need to keep the unique filenames per word using [RDD.distinct()](https://spark.apache.org/docs/2.1.0/api/python/pyspark.html#pyspark.RDD.distinct).  
Then we create a list of filenames. The length of the list is the document frequency DF per word.
From the DF value we can calculate the IDF value as log(18/DF) 

In [22]:
from operator import add
from math import log

fwu_RDD = fw_RDD.distinct() # get unique file/word pairs
wfl_RDD = fwu_RDD.map(lambda fw: (fw[1],[fw[0]])) # create (w,[f]) tuples 
wfL_RDD = wfl_RDD.reduceByKey(add) # concatenate the lists with 'add'
print(wfL_RDD.take(3))

wdf_RDD = wfL_RDD.map(lambda wfl: (wfl[0],len(wfl[1]))) # get the DF replacing the file list with its lenght
print("DF: ",wdf_RDD.take(3))
widf_RDD = wdf_RDD.map(lambda wdf: (wdf[0],log(18/wdf[1]))) # get he IDF by replacing DF with log(19/DF)
print("IDF: ",widf_RDD.take(3))

[('of', ['henry_V', 'king_lear', 'lady_susan', 'macbeth', 'merchant_of_venice', 'midsummer', 'northanger_abbey', 'othello', 'persuasion', 'prideandpredjudice', 'romeo_and_juliet', 'senseandsensibility', 'emma', 'hamlet', 'julius_cesar', 'mansfield_park', 'richard_III', 'tempest']), ('shakespeare', ['henry_V', 'king_lear', 'macbeth', 'merchant_of_venice', 'midsummer', 'northanger_abbey', 'othello', 'romeo_and_juliet', 'senseandsensibility', 'emma', 'hamlet', 'julius_cesar', 'mansfield_park', 'richard_III', 'tempest']), ('henry', ['henry_V', 'merchant_of_venice', 'midsummer', 'northanger_abbey', 'persuasion', 'senseandsensibility', 'emma', 'mansfield_park', 'richard_III'])]
DF:  [('of', 18), ('shakespeare', 15), ('henry', 9)]
IDF:  [('of', 0.0), ('shakespeare', 0.1823215567939546), ('henry', 0.6931471805599453)]
